In [ ]:
%cd graph-enhanced-retrieval-qa
!pwd

/home/sslab/24m0786/graph-enhanced-retrieval-qa
/home/sslab/24m0786/graph-enhanced-retrieval-qa


In [2]:
!nvidia-smi

Sat Aug  2 16:45:01 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L40S                    Off |   00000000:2D:00.0 Off |                    0 |
| N/A   32C    P8             32W /  350W |       4MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import sys
sys.path.append('..')

import torch
from sentence_transformers import SentenceTransformer, util
from src.data_loader import load_dataset, process_sample
from pprint import pprint

dev_dataset = load_dataset('data/raw/dev.json')
sample = dev_dataset[0]
processed_data = process_sample(sample)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

model_path = '/home/sslab/24m0786/.cache/huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181' # <--- REPLACE THIS
print(f"Loading model directly from path: {model_path}")

try:
    model = SentenceTransformer(model_path, device=device)
    print("Model loaded successfully from local path.")
except Exception as e:
    print(f"ERROR: Failed to load model from path '{model_path}'.")
    print(f"Please verify the path is correct and points to the snapshot directory containing the model files.")
    print(f"Error details: {e}")

Using device: cuda
Loading model directly from path: /home/sslab/24m0786/.cache/huggingface/hub/models--BAAI--bge-m3/snapshots/5617a9f61b028005a4858fdac845db406aefb181
Model loaded successfully from local path.


In [4]:
passages_map = processed_data['passages']
corpus_titles = list(passages_map.keys())
corpus_texts = list(passages_map.values())
question = processed_data['question']

question_embedding = model.encode(question, convert_to_tensor=True)
passage_embeddings = model.encode(corpus_texts, convert_to_tensor=True, show_progress_bar=True)

cosine_scores = util.cos_sim(question_embedding, passage_embeddings)[0]
top_results = torch.topk(cosine_scores, k=len(corpus_titles))
ranked_titles = [corpus_titles[i] for i in top_results.indices]

print("\n--- Dense Retrieval Top 5 Ranked Titles ---")
pprint(ranked_titles[:5])

/home/sslab/24m0786/miniconda3/envs/graph_qa/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


--- Dense Retrieval Top 5 Ranked Titles ---
['Polish-Russian War (film)',
 'Xawery Żuławski',
 'Minamoto no Chikako',
 'Alice Washburn',
 'Liberty Ross']
